In [63]:
import urllib.request
import json
import os
import ssl

def create_stop_order(api_key, account_id, instrument, units, price):
    """
    Creates a stop order to buy or sell an instrument at a given price using the Oanda REST API.

    Args:
        instrument (str): The instrument to trade (e.g., 'EUR_USD').
        units (int): The number of units to buy or sell. Positive for a buy order, negative for a sell order.
        price (float): The price at which to trigger the stop order.
    """

    # Oanda API endpoint for creating an order
    url = f"https://api-fxtrade.oanda.com/v3/accounts/{account_id}/orders"

    # Order data
    data = {
        "order": {
            "price": str(price),
            "instrument": instrument,
            "units": str(units),
            "type": "STOP",
            "timeInForce": "GTC"
        }
    }

    # JSON encode the data
    postdata = json.dumps(data).encode('utf-8')

    # Create the request
    req = urllib.request.Request(url, postdata, {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    })

    # Make the request
    try:
        import certifi
        context = ssl.create_default_context(cafile=certifi.where())
        with urllib.request.urlopen(req, context=context) as response:
            response_data = json.loads(response.read().decode('utf-8'))
            print(json.dumps(response_data, indent=2))
    except ImportError:
        print("The 'certifi' package is not installed. Please install it by running '!pip install certifi' in a new cell and then run this cell again.")
    except urllib.error.HTTPError as err:
        print(f"HTTP Error: {err.code} {err.reason}")
        print(err.read().decode('utf-8'))


This notebook will contain a Python function to create a stop order using the Oanda REST API. It will use only Python's built-in libraries to make the API call.

### Example Usage

The following cell demonstrates how to use the `create_stop_order` function. 

**Important:** Before running the example, make sure to replace the placeholder values for `YOUR_API_KEY` and `YOUR_ACCOUNT_ID` in the function with your actual Oanda API key and account ID. For better security, it is recommended to store these credentials as environment variables or in a separate `secrets.json` file.

In [61]:
# Example: Create a stop order to buy 100 units of EUR_USD at 1.1000
# In a real scenario, you would replace the placeholder credentials in the function.
# Load secrets from secrets.json
with open('secrets.json', 'r') as f:
    secrets = json.load(f)

api_key = secrets['api_key']
account_id = secrets['account_id']

create_stop_order(api_key, account_id, "USD_CAD", -160, 1.36100)

{
  "orderCreateTransaction": {
    "id": "688294",
    "accountID": "001-002-6172489-007",
    "userID": 6172489,
    "batchID": "688294",
    "requestID": "133444830488408017",
    "time": "2025-06-26T16:36:41.236385204Z",
    "type": "STOP_ORDER",
    "instrument": "USD_CAD",
    "units": "-160",
    "price": "1.36100",
    "timeInForce": "GTC",
    "triggerCondition": "DEFAULT",
    "triggerMode": "TOP_OF_BOOK",
    "partialFill": "DEFAULT",
    "positionFill": "DEFAULT",
    "reason": "CLIENT_ORDER"
  },
  "relatedTransactionIDs": [
    "688294"
  ],
  "lastTransactionID": "688294"
}


### Cancel Stop Orders

The following function, `cancel_stop_orders`, cancels pending stop orders for a specific instrument. You can specify whether to cancel buy orders, sell orders, or all stop orders for the instrument.

In [53]:
import certifi
def cancel_stop_orders(api_key, account_id, instrument, direction):
    """
    Cancels pending stop orders for a given instrument.

    Args:
        api_key (str): Your Oanda API key.
        account_id (str): Your Oanda account ID.
        instrument (str): The instrument to cancel orders for (e.g., 'EUR_USD').
        direction (int): The direction of orders to cancel:
                         -1 to cancel sells, 1 to cancel buys, 0 to cancel all.
    """
    # Oanda API endpoint for getting pending orders
    get_orders_url = f"https://api-fxtrade.oanda.com/v3/accounts/{account_id}/pendingOrders"

    # Headers for the request
    headers = {
        "Authorization": f"Bearer {api_key}"
    }

    try:
        # Get the list of pending orders
        req = urllib.request.Request(get_orders_url, headers=headers)
        context = ssl.create_default_context(cafile=certifi.where())
        with urllib.request.urlopen(req, context=context) as response:
            pending_orders_data = json.loads(response.read().decode('utf-8'))

        # Filter for stop orders for the given instrument
        for order in pending_orders_data.get('orders', []):
            if order.get('type') == 'STOP' and order.get('instrument') == instrument:
                order_units = int(order.get('units', 0))
                cancel = False
                if direction == 0:
                    cancel = True
                elif direction == 1 and order_units > 0:
                    cancel = True
                elif direction == -1 and order_units < 0:
                    cancel = True

                if cancel:
                    order_id = order['id']
                    cancel_order_url = f"https://api-fxtrade.oanda.com/v3/accounts/{account_id}/orders/{order_id}/cancel"
                    cancel_req = urllib.request.Request(cancel_order_url, headers=headers, method='PUT')
                    with urllib.request.urlopen(cancel_req, context=context) as cancel_response:
                        cancel_data = json.loads(cancel_response.read().decode('utf-8'))
                        print(f"Canceled order {order_id}:")
                        print(json.dumps(cancel_data, indent=2))

    except ImportError:
        print("The 'certifi' package is not installed. Please install it by running '!pip install certifi' in a new cell and then run this cell again.")
    except urllib.error.HTTPError as err:
        print(f"HTTP Error: {err.code} {err.reason}")
        print(err.read().decode('utf-8'))


In [62]:
with open('secrets.json', 'r') as f:
    secrets = json.load(f)

api_key = secrets['api_key']
account_id = secrets['account_id']

cancel_stop_orders(api_key, account_id, "USD_CAD", 0)  # Cancel all stop orders for USD_CAD

Canceled order 688294:
{
  "orderCancelTransaction": {
    "id": "688295",
    "accountID": "001-002-6172489-007",
    "userID": 6172489,
    "batchID": "688295",
    "requestID": "169473627531128193",
    "time": "2025-06-26T16:36:47.452237688Z",
    "type": "ORDER_CANCEL",
    "orderID": "688294",
    "reason": "CLIENT_REQUEST"
  },
  "relatedTransactionIDs": [
    "688295"
  ],
  "lastTransactionID": "688295"
}


### Get Instrument Position

The following function, `get_instrument_position`, retrieves the number of units (shares) you hold for a given instrument.

In [ ]:
def get_instrument_position(api_key, account_id, instrument):
    """
    Retrieves the number of units for a given instrument.

    Args:
        api_key (str): Your Oanda API key.
        account_id (str): Your Oanda account ID.
        instrument (str): The instrument to check (e.g., 'EUR_USD').

    Returns:
        int: The number of units of the instrument. Positive for long, negative for short, 0 for no position.
    """
    get_positions_url = f"https://api-fxtrade.oanda.com/v3/accounts/{account_id}/openPositions"

    headers = {
        "Authorization": f"Bearer {api_key}"
    }

    try:
        req = urllib.request.Request(get_positions_url, headers=headers)
        context = ssl.create_default_context(cafile=certifi.where())
        with urllib.request.urlopen(req, context=context) as response:
            positions_data = json.loads(response.read().decode('utf-8'))

        for position in positions_data.get('positions', []):
            if position.get('instrument') == instrument:
                long_units = int(position.get('long', {}).get('units', 0))
                short_units = int(position.get('short', {}).get('units', 0))
                return long_units - short_units
        
        return 0 # No position found for the instrument

    except ImportError:
        print("The 'certifi' package is not installed. Please install it by running '!pip install certifi' in a new cell and then run this cell again.")
        return None
    except urllib.error.HTTPError as err:
        print(f"HTTP Error: {err.code} {err.reason}")
        print(err.read().decode('utf-8'))
        return None
